In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
def GKHV(o,h,l,c):

    volatility = (1/2)*((np.log(h/l))**2) + (2*(np.log(2))-1)*((np.log(c/o))**2)

    return volatility

In [3]:
def RS(o,h,l,c,prev_c):

    u = np.log(h/o)
    c = np.log(c/o)
    d = np.log(l/o)


    volatility = u*(u-c) + d*(d-c)

    return volatility


In [4]:
data_path = '../data/SL20 Two years to Narada.xls'

data = pd.read_excel(data_path, index_col= False)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [5]:
def split_(stock_name):
    return stock_name.split('.')[0]

In [6]:
data['SECURITYCODE'] = data['SECURITYCODE'].apply(lambda x : split_(x))

In [7]:
data = data[['SECURITYCODE','OPENINGPRICE','HIGHPX', 'LOWPX', 'CLOSINGPRICE', 'TRADEDATE', 'UNIX_TS']].dropna(axis= 0, how = 'any')
# data

In [8]:
data = data.sort_values(by = 'UNIX_TS')

In [9]:
symbols = list(data.SECURITYCODE.unique())
symbol_df = data[data.SECURITYCODE == symbols[0]].copy()
symbol_df.head(2)

,SECURITYCODE,OPENINGPRICE,HIGHPX,LOWPX,CLOSINGPRICE,TRADEDATE,UNIX_TS
17913,DFCC,61.0,61.0,59.7,59.8,2022-02-08,1644278400
17842,DFCC,59.0,60.0,58.8,58.8,2022-02-09,1644364800


In [10]:
symbol_df['gkhv'] = symbol_df.apply(lambda row: GKHV(row['OPENINGPRICE'],row['HIGHPX'],row['LOWPX'],row['CLOSINGPRICE']), axis = 1)
gkhv = np.sqrt(symbol_df.gkhv.mean())

symbol_df['prev_c'] = symbol_df['CLOSINGPRICE'].shift(1)
symbol_df['rs'] = symbol_df.apply(lambda row: RS(row['OPENINGPRICE'],row['HIGHPX'],row['LOWPX'],row['CLOSINGPRICE'], row['prev_c']), axis = 1)
rs = np.sqrt(symbol_df.rs.mean())

symbol_df['norm_o'] = symbol_df['OPENINGPRICE']/symbol_df['CLOSINGPRICE'].shift(1)
symbol_df['norm_c'] = symbol_df['CLOSINGPRICE']/symbol_df['OPENINGPRICE']
k = (0.34/(1.34 + ((len(symbol_df)+1)/(len(symbol_df)-1))))

yangzhang = np.sqrt((symbol_df['norm_o'].std()**2) + (k*symbol_df['norm_c'].std()**2) + (1-k)*symbol_df['rs'].mean())

In [11]:
gkhv, rs, yangzhang

(0.03527880832344654, 0.028366201894224046, 0.03309326173659469)

In [12]:
symbol_df.to_excel('dfcc.xlsx')

In [13]:
symbol_df

,SECURITYCODE,OPENINGPRICE,HIGHPX,LOWPX,CLOSINGPRICE,TRADEDATE,UNIX_TS,gkhv,prev_c,rs,norm_o,norm_c
17913,DFCC,61.0,61.0,59.7,59.8,2022-02-08,1644278400,0.000385,NaN,0.000036,NaN,0.980328
17842,DFCC,59.0,60.0,58.8,58.8,2022-02-09,1644364800,0.000209,59.8,0.000340,0.986622,0.996610
17802,DFCC,59.0,59.2,58.5,59.0,2022-02-10,1644451200,0.000071,58.8,0.000084,1.003401,1.000000
17779,DFCC,58.7,59.0,58.7,59.0,2022-02-11,1644537600,0.000023,59.0,0.000000,0.994915,1.005111
17729,DFCC,59.1,59.1,58.6,58.9,2022-02-14,1644796800,0.000041,59.0,0.000043,1.001695,0.996616
...,...,...,...,...,...,...,...,...,...,...,...,...
156,DFCC,73.5,74.0,72.0,72.5,2024-01-31,1706659200,0.000448,73.0,0.000282,1.006849,0.986395
122,DFCC,74.0,75.0,74.0,75.0,2024-02-01,1706745600,0.000160,72.5,0.000000,1.020690,1.013514
100,DFCC,76.0,76.9,75.0,75.4,2024-02-02,1706832000,0.000337,75.0,0.000302,1.013333,0.992105
56,DFCC,75.0,78.1,75.0,77.9,2024-02-06,1707177600,0.001376,75.4,0.000104,0.994695,1.038667


In [15]:
pathh = '../data/stock_data.csv'
stock_data = pd.read_csv(pathh)

In [16]:
stock_data

,symbol,name,buisnesssummary,gics_#NAME?,gics_45103010 - Application Software,gics_Automobiles & Components,gics_Banks,gics_Banks Finance & Insurance,gics_Capital Goods,gics_Capital Goods.,...,gics_Power and Energy,gics_Real Estate,gics_Real Estate (6010),gics_Real Estate Development,gics_Retailing,gics_Services,gics_Telecommunication Services,gics_Trading,gics_Transpotation,gics_Utilities
0,ABAN,ABANS ELECTRICALS PLC,Manufacturing & Assembling Household Electric ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AFSL,ABANS FINANCE PLC,unknown,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AEL,ACCESS ENGINEERING PLC,unknown,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,ACL,ACL CABLES PLC,Manufacturing and Marketing of Cables and Cond...,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,APLA,ACL PLASTICS PLC,Manufacturing of Cable grade PVC Compound.,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,VLL,VIDULLANKA PLC,Produce And Transmit Electrical Energy To The ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
278,VPEL,VALLIBEL POWER ERATHNA PLC,Generate and supply electric power to the nati...,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
279,VONE,VALLIBEL ONE PLC,unknown,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
280,VFIN,VALLIBEL FINANCE PLC,unknown,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
gics = stock_data.drop(['name','buisnesssummary', 'symbol'], axis = 1)
# gics.head()

In [18]:
gics_code = gics.idxmax(axis = 1).values
gics_code


array(['gics_Consumer Durables & Apparel', 'gics_Diversified Financials',
       'gics_Capital Goods', 'gics_Capital Goods', 'gics_Materials',
       'gics_Materials', 'gics_FOOD BEVERAGE & TOBACCO', 'gics_Materials',
       'gics_Consumer Services', 'gics_Food Beverage & Tobacco',
       'gics_Capital Goods', 'gics_Diversified Financials',
       'gics_Capital Goods', 'gics_Materials', 'gics_Banks',
       'gics_Insurance', 'gics_Insurance',
       'gics_Consumer Durables & Apparel',
       'gics_Consumer Durables & Apparel', 'gics_Diversified Financials',
       'gics_Consumer Services', 'gics_Insurance',
       'gics_Diversified Financials', 'gics_Retailing',
       'gics_FOOD & STAPLES RETAILING', 'gics_Real Estate',
       'gics_Retailing', 'gics_Closed End Fund',
       'gics_Household & Personal Products',
       'gics_FOOD BEVERAGE & TOBACCO', 'gics_FOOD BEVERAGE & TOBACCO',
       'gics_Consumer Services', 'gics_Consumer Services',
       'gics_Diversified Financials', 'gics_C

In [19]:
stock_data_new = stock_data[['symbol','name','buisnesssummary']].copy()
stock_data_new['gics_code'] = gics_code
# stock_data_new

In [20]:
stock_data_new['gics_code'] = stock_data_new['gics_code'].apply(lambda x : x.split('_')[1])

In [21]:
stock_data_new.head()

,symbol,name,buisnesssummary,gics_code
0,ABAN,ABANS ELECTRICALS PLC,Manufacturing & Assembling Household Electric ...,Consumer Durables & Apparel
1,AFSL,ABANS FINANCE PLC,unknown,Diversified Financials
2,AEL,ACCESS ENGINEERING PLC,unknown,Capital Goods
3,ACL,ACL CABLES PLC,Manufacturing and Marketing of Cables and Cond...,Capital Goods
4,APLA,ACL PLASTICS PLC,Manufacturing of Cable grade PVC Compound.,Materials


In [22]:
stock_data_new.gics_code.value_counts()

Diversified Financials                40
Consumer Services                     35
Food Beverage & Tobacco               35
Capital Goods                         28
Materials                             19
Real Estate                           16
Retailing                             14
Consumer Durables & Apparel           13
Banks                                 12
FOOD BEVERAGE & TOBACCO               10
Insurance                             10
Health Care Equipment & Services       7
Commercial & Professional Services     7
Utilities                              7
Investment Banking & Brokerage         2
FOOD & STAPLES RETAILING               2
Telecommunication Services             2
#NAME?                                 2
Energy                                 2
Household & Personal Products          2
Diversified Financial Services         1
Consumer Finance                       1
Trading                                1
Real Estate (6010)                     1
Real Estate Deve

In [23]:
stock_data_new.head()

,symbol,name,buisnesssummary,gics_code
0,ABAN,ABANS ELECTRICALS PLC,Manufacturing & Assembling Household Electric ...,Consumer Durables & Apparel
1,AFSL,ABANS FINANCE PLC,unknown,Diversified Financials
2,AEL,ACCESS ENGINEERING PLC,unknown,Capital Goods
3,ACL,ACL CABLES PLC,Manufacturing and Marketing of Cables and Cond...,Capital Goods
4,APLA,ACL PLASTICS PLC,Manufacturing of Cable grade PVC Compound.,Materials


In [27]:
stock_data_new.to_excel('stock_data_w_gics.xlsx')

In [26]:
gics_groups = stock_data_new.groupby(by = 'gics_code')
gics_groups.groups

{'#NAME?': [169, 187], '45103010 - Application Software': [127], 'Automobiles & Components': [141], 'Banks': [14, 74, 90, 109, 123, 192, 193, 208, 244, 252, 254, 275], 'Banks Finance & Insurance': [120], 'Capital Goods': [2, 3, 10, 12, 37, 52, 69, 86, 115, 117, 133, 140, 151, 152, 157, 160, 164, 171, 172, 197, 211, 213, 216, 231, 240, 247, 264, 273], 'Capital Goods.': [159], 'Closed End Fund': [27], 'Commercial & Professional Services': [61, 70, 99, 105, 149, 199, 206], 'Consumer Durables & Apparel': [0, 17, 18, 34, 73, 112, 113, 116, 143, 220, 223, 236, 267], 'Consumer Finance': [251], 'Consumer Services': [8, 20, 31, 32, 38, 43, 59, 81, 82, 95, 103, 114, 118, 122, 124, 132, 134, 175, 177, 205, 209, 210, 215, 218, 221, 239, 246, 255, 258, 259, 261, 262, 263, 269, 271], 'Diversified Financial Services': [153], 'Diversified Financials': [1, 11, 19, 22, 33, 41, 42, 46, 51, 57, 60, 68, 76, 87, 88, 91, 101, 104, 106, 128, 146, 147, 158, 165, 168, 180, 182, 185, 189, 195, 201, 203, 232, 234

In [28]:
data_path = '../data/SL20 Two years to Narada.xls'

data = pd.read_excel(data_path, index_col= False)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [31]:
data.TRADEDATE.min(), data.TRADEDATE.max()

(Timestamp('2022-02-08 00:00:00'), Timestamp('2024-02-07 00:00:00'))